In [1]:
import sys, os, glob
from os.path import join
import pypillometry as pp
import pandas as pd
import numpy as np
import pylab as plt
import scipy.io
import scipy.signal

os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [2]:
##
# Define functions for later use
##
import matplotlib.pyplot
# function to apply bandpass filter
def bandPass(signal):
    fs = 1200.0
    lowcut = 0.1
    highcut = 40.0

    nyq = 0.5*fs
    low = lowcut/nyq
    high = highcut/nyq

    order = 2

    b,a = scipy.signal.butter(order, [low, high], 'bandpass', output='ba') # analog=True or analog=False
    y = scipy.signal.filtfilt(b, a, signal, axis=-1)
    
    # trying to keep signal at value location
    # zi = scipy.signal.lfilter_zi(b, a); 
    # y, zo = scipy.signal.lfilter(b, a, signal, zi=zi*signal[0])
    # y = scipy.signal.lfilter(b, a, signal, axis=0, zi=None)
    
    return y

# function to load a mat file and store it in pd dataframe
def load_mat(matLoc, checkIntPerc = False, data_name = 'Data'):

    # vars for voltage correction
    maxvolt = 5
    minvolt = -5
    
    mat = scipy.io.loadmat(matLoc)

    concat_df = pd.DataFrame(data=[])

    interpol_vals = [0]*24
    raw_vals = [0]*24
    
    for b in range(24):
        df = pd.DataFrame(data=pd.Series(mat['tskData'][0][0][7][0][b][0]), columns=['time'])
        df['pupil'] = mat['tskData'][0][0][6][0][b][3]    
        df['block'] = b+1

        # interpolate voltages to sizes
        df.pupil = (df.pupil-minvolt)/(maxvolt-minvolt)
        
        # due to continuous values, replace mising data with 0
        df['pupil'].mask(df['pupil'] <= 0.01, 0, inplace=True)

        # apply bandpass filter
        # df.pupil = bandPass(df.pupil)

        ## ToDo check for percentage of interpolated data
        if checkIntPerc:
            # print(b)
            int_d, raw_d = int_perc_check(df)
            interpol_vals[b] = int_d
            raw_vals[b] = raw_d

            if ((int_d/raw_d)<=0.3): # see https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6535748/
                # concatenate over experimental blocks
                concat_df = pd.concat([concat_df, df], ignore_index=True)
        else:
            concat_df = pd.concat([concat_df, df], ignore_index=True)
        
    # print(interpol_vals) # print values 
    
    # Pass the x and y cordinates of the bars to the
    # function. The label argument gives a label to the data.
    calc_data = [0]*24
    for i in range(24):
        calc_data[i] = interpol_vals[i]/raw_vals[i]

    # plot:
    matplotlib.pyplot.bar(range(1,25),calc_data, label=data_name)
    matplotlib.pyplot.legend()
    
    # The following commands add labels to our figure.
    matplotlib.pyplot.xlabel('Blocks')
    matplotlib.pyplot.ylabel('percent')
    matplotlib.pyplot.axhline(0.3, color='red')
    matplotlib.pyplot.title('Interpolated data in blocks from ' + data_name)
    matplotlib.pyplot.savefig(datapath+'pp_dfs/pics_interpolated/'+data_name+'.png', dpi=400)    
    matplotlib.pyplot.show()

    return concat_df


## function to check for percentage of interpolated data (for plotting histogram later)
def int_perc_check(pp_data):
    data = pp.PupilData(pp_data.pupil, time=pp_data.time, name='test', sampling_rate=1200, fill_time_discontinuities=False)
    pp_preprocd = data.blinks_detect(winsize = 150,min_duration=10,strategies=["zero","velocity"],
                       vel_onset=-0, vel_offset=0,
                       min_onset_len=2, min_offset_len=2)\
                .blinks_merge(distance=100)\
                .blinks_interpolate(winsize=10, margin=(50,150), vel_onset=-0, vel_offset=0)
                #.blinks_interp_mahot(winsize=10, margin=(50,150), vel_onset=-0, vel_offset=0) ## currently breaks 

    n_inter = sum(pp_preprocd.interpolated_mask) # number of interpolated points
    n_raw = len(pp_preprocd.sy) # number of raw datapoints
    # print(str(n_inter/n_raw)) # print percent of interpolated points
    return[n_inter,n_raw]

# function to load in multiple datasets
def load_PP_data(dataLs, saveBool = True, checkIntPerc = False):

    datasets = [None]*len(dataLs)
    
    for idx,d in enumerate(dataLs):
        # load data from mat file
        pupil_df = load_mat(datapath+d+'/main/'+'main_eye_'+d+'.mat', checkIntPerc, d)

        # load additional behavioural data from csv
        behav_df =pd.read_csv(datapath+d+'/main/'+'main_behdf_'+d+'.csv')


        if pupil_df.empty != True:
            # create pupilData based on combination of pupil and behavioural data
            pp_Data = pp.PupilData(pupil_df.pupil, time=pupil_df.time, name=d, sampling_rate=1200, fill_time_discontinuities=False ,event_onsets=behav_df.timing_meg, event_labels=behav_df.block) # standard fill_time_discontinuities=True
    
        # save PupilData in array
        datasets[idx] = pp_Data

        # save some ram space
        pp_Data = []

    if saveBool:
        # save pupilData datasets on disk
        for ds in datasets:
            if ds != []:
                fname=os.path.join(datapath+'pp_dfs/', ds.name+".pd")
                ds.write_file(fname)
    
    return datasets

## Loading raw data

In [ ]:
datapath = '/Users/scanlab/Documents/internship_luca/pupildata/' # os.getcwd()

data2use = ['sub-004', 'sub-005', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011']#, 'sub-013', 'sub-014']

# load in pupil data and save as pypillometry file on disk
load_PP_data(data2use, False, True)

## Preprocessing

In [ ]:
datapath = '/Users/scanlab/Documents/internship_luca/pupildata/pp_dfs/'

# load all filenames in `datapath` that end with `.pd`
pd_files=[fname for fname in os.listdir(datapath) if fname.endswith(".pd")]
datasets=[]
for fname in pd_files:
    fname=os.path.join(datapath, fname)
    d=pp.PupilData.from_file(fname)
    datasets.append(d)

# load all raw datasets
exclude=['sub-001','sub-002','sub-012', 'sub-013', 'sub-014'] ## these subject did not have usable data

# datasets are stored in a dict structure
datasets={d.name.split("_")[0]:d for d in datasets if d.name not in exclude}

In [ ]:
# specify pre-processing parameters per subj
default_param={"min_duration":10,    # min duration of a blink
               "min_offset_len":2,   # offset/onset-length for blink-detection
               "min_onset_len":3,
               "vel_onset":-0,       # velocity thresholds for onset and offset
               "vel_offset":0,
               "strategies":["zero","velocity"],  # strategies for blink-detection
              "distance":100,        # minimum distance between two blinks
              "margin":(50,150),     # margins for Mahot algorithm
              "cutoff":5,            # lowpass-filter cutoff (Hz)
              "downsample":50}       # downsample to XX Hz

# create dict with parameters per subject
# all subjects get the same set of default parameters initially
params={subj:default_param.copy() for subj in datasets.keys()}

In [ ]:
## fine-tuning of the parameters per subject
# subj = "sub-001"
# No pupil data!
# IndexError: index -1 is out of bounds for axis 0 with size 0
# subj = "sub-002"
# No pupil data!
# IndexError: index -1 is out of bounds for axis 0 with size 0
subj = "sub-004"
# Writes File
subj = "sub-005"
# Writes File
subj = "sub-007"
# Writes File
subj = "sub-008"
# Writes File
subj = "sub-009"
# Writes File
subj = "sub-010"
# Writes File
#params[subj]["vel_onset"]=-10
subj = "sub-011"
# Writes File
# subj = "sub-012"
# No pupil data!
# IndexError: index -1 is out of bounds for axis 0 with size 0
subj = "sub-013"
# Writes File
subj = "sub-014"
# Writes File

## notes for preprocessing
#### subject 4
- spike at -2.282
- spike at -2.273
- spike at -2.266
- spike at -2.256
- spike at -2.236
- spike at -2.225
#### subject 5
- very noisy
- many spikes in beginning & end
- data of blocks blur together
#### subject 7
- clean data with some eyeblinks
- interpolated data seems quite good
#### subject 8
- clean data
- not many eyeblinks in the first place
- => therefore: some noise in interpolated data might still be existend
#### subject 9
- overal quite clean data
- spike at -2.565
- spike at -2.548
#### subject 10
- clean data, little blinks/artefacts
- spike at 0.1525
- spike at 0.161
- spike at end (0.2)
#### subject 11
- very clean, little eyeblinks/artefacts
- many spikes between 0,27 to 0.28
#### subject 13
- many eye blinks
- start from 0?
- no visible spikes
#### subject 14
- moderate number of eyeblinks
- much missing data!
- interpolated data makes many "jumpes"
- very noisy

In [ ]:
## run pre-proc pipeline on all subjects and produce PDFs for inspection
# add downsampling?
preprocs={}
# go trough datasets
for subj,d in datasets.items():
    print(d.name)
    pars=params[subj]
    # apply predefined parameters
    # dp=d.reset_time()\
    dp=d.downsample(400)\
            .blinks_detect(winsize = 150,min_duration=pars["min_duration"],strategies=pars["strategies"],
                       vel_onset=pars["vel_onset"], vel_offset=pars["vel_offset"],
                       min_onset_len=pars["min_onset_len"], min_offset_len=pars["min_offset_len"])\
            .blinks_merge(distance=pars["distance"])\
            .blinks_interp_mahot(margin=pars["margin"], vel_onset=pars["vel_onset"], vel_offset=pars["vel_offset"])\
            .lowpass_filter(cutoff=pars["cutoff"]).downsample(40)#.reset_time(inplace=True)
    # plot preprocessed data and save as pdf or png
    # d.reset_time(inplace=True)
    plt.figure(figsize=(15,5))
    d.plot((d.tx.min(),d.tx.min()+100), units='ms', highlight_blinks=False)
    dp.plot((d.tx.min(),d.tx.min()+100), units='ms', highlight_blinks=False)
    plt.savefig('/Users/scanlab/Documents/internship_luca/pupildata/pp_dfs/pics/'+subj+'.png')
    # d.plot_segments(overlay=dp, pdffile="/home/luca/Documents/internship/pupildata/pp_dfs/pics/%s.pdf"%d.name,figsize=(40,6), ylim=(dp.sy.min(), dp.sy.max()))
    # save preprocessed dataset
    preprocs[subj]=dp

In [ ]:
# save data in individual csv files
os.makedirs('/Users/scanlab/Documents/internship_luca/pupildata/pp_dfs/preprocs/', exist_ok=True)  
for subj, d in preprocs.items():
    if subj not in exclude:
        prepro_df =  pd.DataFrame(d.sy, columns = ['pupil'])
        prepro_df['time'] =  d.tx
        prepro_df.to_csv('/Users/scanlab/Documents/internship_luca/pupildata/pp_dfs/preprocs/40Hz/'+subj+'.csv') 